In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=83e1703598b599c05ea5c3764dff848d5a03c9545b83dbde51464e051bdf38c3
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


# Создаём точку входа в Spark

In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F


spark = SparkSession.builder\
        .master('local[2]')\
        .appName('Lesson_2')\
        .config('spark.ui.port', '4050')\
        .config('spark.executor.instances', 2)\
        .config('spark.executor.memory', '5g')\
        .config('spark.executor.cores', 2)\
        .getOrCreate()

sc = spark.sparkContext

In [3]:
spark

# DataFrame
Теперь мы знаем про Dataframe. Нужно сделать практически всё то же самое, но используя датафрейм.

# Самостоятельная работа

Требуется выяснить:
1. Какое соотношение сторон экрана телефона самое популярное,
2. Плотность пикселей у экрана.

Всего 8 баллов. 


## Считывание данных
Данные взяты отсюда: https://www.kaggle.com/iabhishekofficial/mobile-price-classification
Скачиваем и копируем в папку с нотебуком

Внутри содержится следующая информация:

* id: ID
* battery_power: Total energy a battery can store in one time (mAh)
* blue: Support bluetooth or not
* clock_speed: Speed at which microprocessor executes instructions
* dual_sim: Support dual sim or not
* fc: Front Camera mega pixels
* four_g: Support 4G or not
* int_memory: Internal Memory (GB)
* m_dep: Mobile Depth (cm)
* mobile_wt: Weight of mobile phone
* n_cores: Number of cores of processor
* pc: Primary Camera mega pixels
* px_height: Pixel Resolution Height
* px_width: Pixel Resolution Width
* ram: Random Access Memory (MB)
* sc_h: Screen Height of mobile (cm)
* sc_w: Screen Width of mobile (cm)
* talk_time: Time that a single battery charge will last
* three_g: Support 3G or not
* touch_screen: Has touch screen or not
* wifi: Support wifi or not

In [4]:
import requests

import pandas as pd

# Считывание данных

In [ ]:
# Прочитайте содержимое каждого файла в датафрейм и выведите первые несколько строк
# Применив cast к колонке можно изменить тип данных (F.col().cast(IntegerType()).alias())
# Приведите все данные к правильному типу, либо считайе сразу верно

In [5]:
train_response = requests.get('https://drive.google.com/file/d/13lRX0J_fDADB1g6OTJLH5hS3gKEymX9J')
with open('train.csv', 'wb') as file:
    file.write(train_response.content)
train = spark.read.csv('train.csv', header=True)

test_response = requests.get('https://drive.google.com/file/d/1U3wGRi5RTpSIdCCr2_N7zTPgUtwEKbH0')
with open('test.csv', 'wb') as file:
    file.write(test_response.content)
test = spark.read.csv('test.csv', header=True)

In [6]:
train.show()

+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+-----------+
|battery_power|blue|clock_speed|dual_sim| fc|four_g|int_memory|m_dep|mobile_wt|n_cores| pc|px_height|px_width| ram|sc_h|sc_w|talk_time|three_g|touch_screen|wifi|price_range|
+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+-----------+
|          842|   0|        2.2|       0|  1|     0|         7|  0.6|      188|      2|  2|       20|     756|2549|   9|   7|       19|      0|           0|   1|          1|
|         1021|   1|        0.5|       1|  0|     1|        53|  0.7|      136|      3|  6|      905|    1988|2631|  17|   3|        7|      1|           1|   0|          2|
|          563|   1|        0.5|       1|  2|     1|        41|  0.9|      145|      5|  6|     1263|    1716|2603|  11|   2|     

## Объединим train и test
Найти нужную функцию можно [здесь](https://spark.apache.org/docs/2.4.0/api/python/pyspark.sql.html)

PS: нужно сделать средствами pyspark pd.concat([train, test,], axis=0)

In [ ]:
# Объедините train и test (1 балла)

In [7]:
data = train.drop('price_range').unionByName(test.drop('id'))

In [8]:
data.count()

3000

In [9]:
train = train.drop('price_range')
test = test.drop('id')

pandas_train = train.toPandas()
pandas_test = test.toPandas()

merged_pandas_df = pd.concat([pandas_train, pandas_test], axis=0)

spark_df = spark.createDataFrame(merged_pandas_df)

In [10]:
spark_df.printSchema()

root
 |-- battery_power: string (nullable = true)
 |-- blue: string (nullable = true)
 |-- clock_speed: string (nullable = true)
 |-- dual_sim: string (nullable = true)
 |-- fc: string (nullable = true)
 |-- four_g: string (nullable = true)
 |-- int_memory: string (nullable = true)
 |-- m_dep: string (nullable = true)
 |-- mobile_wt: string (nullable = true)
 |-- n_cores: string (nullable = true)
 |-- pc: string (nullable = true)
 |-- px_height: string (nullable = true)
 |-- px_width: string (nullable = true)
 |-- ram: string (nullable = true)
 |-- sc_h: string (nullable = true)
 |-- sc_w: string (nullable = true)
 |-- talk_time: string (nullable = true)
 |-- three_g: string (nullable = true)
 |-- touch_screen: string (nullable = true)
 |-- wifi: string (nullable = true)



## Рассчитайте соотношение сторон телефона и экрана

In [ ]:
# Выведите отсортированное распределение плотности пикселей (1 балла)
# в пикселях на дюйм

In [11]:
data = data.withColumn('pixel_density', 
                       F.round(F.sqrt((data.px_height ** 2 + data.px_width ** 2) / 
                                      (data.sc_h ** 2 + data.sc_w ** 2) ** 0.5)))

data = data.withColumn('aspect_ratio', F.round(data['sc_w'] / data['sc_h'], 2))

data.show(5)

+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+-------------+------------+
|battery_power|blue|clock_speed|dual_sim| fc|four_g|int_memory|m_dep|mobile_wt|n_cores| pc|px_height|px_width| ram|sc_h|sc_w|talk_time|three_g|touch_screen|wifi|pixel_density|aspect_ratio|
+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+-------------+------------+
|          842|   0|        2.2|       0|  1|     0|         7|  0.6|      188|      2|  2|       20|     756|2549|   9|   7|       19|      0|           0|   1|        224.0|        0.78|
|         1021|   1|        0.5|       1|  0|     1|        53|  0.7|      136|      3|  6|      905|    1988|2631|  17|   3|        7|      1|           1|   0|        526.0|        0.18|
|          563|   1|        0.5|       1|  2|     1|   

## Сохранение


In [ ]:
# Сохраните результат в csv sep=';', encoding='cp1251'
# с колонками id, плотность пикселей и общей инфой в формате "2 ядра 2 гига"

In [12]:
data.select(
    F.monotonically_increasing_id(), 
    'pixel_density', 
    F.concat_ws(' ',
        F.col('n_cores'), 
        F.lit('ядра')).alias('desc'))\
.write.option('encoding', 'cp1252').csv('results.csv', mode='overwrite', sep=';')